# 第20章 并发执行器

## 20.1 并发网络下载

### 20.1.1 依序下载版本

In [2]:
import time
from pathlib import Path
from typing import Callable

import httpx

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

# 存放国旗图像的网络资源
BASE_URL = 'https://www.fluentpython.com/data/flags'
# 保存图像的本地目录
DEST_DIR = Path('downloaded')                         

def save_flag(img: bytes, filename: str) -> None: 
    (DEST_DIR / filename).write_bytes(img)

def get_flag(cc: str) -> bytes:  # <6>
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    # 建立网络响应
    resp = httpx.get(url, timeout=6.1, follow_redirects=True)  
    resp.raise_for_status()
    return resp.content

def download_many(cc_list: list[str]) -> int:
    # 依序下载
    for cc in sorted(cc_list):                 
        image = get_flag(cc)
        save_flag(image, f'{cc}.gif')
        print(cc, end=' ', flush=True)         
    return len(cc_list)

def main(downloader: Callable[[list[str]], int]) -> None:
    # 创建目录
    DEST_DIR.mkdir(exist_ok=True)                          
    # 记录并报告downloader运行时间
    t0 = time.perf_counter()                               
    count = downloader(POP20_CC)
    elapsed = time.perf_counter() - t0
    print(f'\n{count} downloads in {elapsed:.2f}s')

In [3]:
main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 downloads in 14.90s


### 20.1.2 使用concurrent.futures模块下载

In [4]:
from concurrent import futures

def download_one(cc: str):
    # 下载单个图像
    image = get_flag(cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc


def download_many(cc_list: list[str]) -> int:
    # 使用ThreadPoolExcutor作为上下文管理器
    with futures.ThreadPoolExecutor() as executor:  
        # 在多个线程中并发调用
        res = executor.map(download_one, sorted(cc_list))  

    return len(list(res))

In [5]:
main(download_many)

ID ET MX BD DE PH US IN JP NG TR VN RU FR PKBR  CN CD IR EG 
20 downloads in 0.99s


### 20.1.3 future对象

- future对象不应自己动手创建，只能由并发框架实例化。future对象表示终将运行的操作，必须排期运行。
- `.done()`方法：该方法不阻塞，返回一个布尔值，指明future对象包装的可调用对象是否已经执行。
- `.result()`方法：当future对象运行结束后，返回可调用对象的结果，或者重新抛出执行可调用对象时抛出的异常。
- `Executor.map`方法：返回一个迭代器，迭代器的`__next__`方法调用各个future对象的`result`方法。

## 20.2 使用concurrent.futures启动进程

In [13]:
from time import sleep, strftime
from concurrent import futures

def display(*args):
    # 打印时间戳
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)

def loiter(n):  
    # 在开始时显示一个消息，休眠n秒，在结束时再显示一个消息
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n * 10

def main():
    display('Script starting.')
    # 创建进程池，3个worker
    executor = futures.ThreadPoolExecutor(max_workers=3)  
    # 把5个任务提交给executor
    results = executor.map(loiter, range(5))  
    display('results:', results) 
    display('Waiting for individual results:')
    for i, result in enumerate(results):  
        display(f'result {i}: {result}')

In [14]:
main()

[14:58:48] Script starting.
[14:58:48] loiter(0): doing nothing for 0s...
[14:58:48] loiter(0): done.
[14:58:48] 	loiter(1): doing nothing for 1s...
[14:58:48] 		loiter(2): doing nothing for 2s...
[14:58:48] 			loiter(3): doing nothing for 3s...
[14:58:48] results: <generator object Executor.map.<locals>.result_iterator at 0x000001EC5608ED50>
[14:58:48] Waiting for individual results:
[14:58:48] result 0: 0
[14:58:49] 	loiter(1): done.
[14:58:49] 				loiter(4): doing nothing for 4s...
[14:58:49] result 1: 10
[14:58:50] 		loiter(2): done.
[14:58:50] result 2: 20
[14:58:51] 			loiter(3): done.
[14:58:51] result 3: 30
[14:58:53] 				loiter(4): done.
[14:58:53] result 4: 40


## 20.3 杂谈

- 并发是计算机科学中最难的概念之一（通常最好别去招惹它）。
- 千万不要自己管理线程和锁。
- `concurrent.future`包，把线程、进程和队列视作服务的基础设施，不用自己动手直接处理。